In [ ]:
import os
import requests

from path import Path
import fastai.vision.all as V
from fastai.vision.all import L

# Bing configuration
AZURE_SEARCH_KEY = os.environ['AZURE_SEARCH_KEY']
BING_API_ENDPOINT = "https://api.bing.microsoft.com/v7.0/images/search"

### Project categories

In [ ]:
path = Path('images')
categories = 'la-pavoni-europiccola', 'niche-zero', 'moccamaster'

### Functions

In [ ]:
def search_images_bing(key: str, term: str) -> L:
    headers = {'Ocp-Apim-Subscription-Key': key}
    params = {'q': term, 'textDecorations': True, 'textFormat': 'HTML'}
    response = requests.get(BING_API_ENDPOINT, headers=headers, params=params)
    response.raise_for_status()
    search_results = L(response.json()['value'])
    return search_results


def create_dataset(path: Path, categories: iter) -> None:
    if not path.exists():
        for category in categories:
            dest = (path/category)
            os.makedirs(dest, exist_ok=True)
            results = search_images_bing(AZURE_SEARCH_KEY, f'{category}'.replace('-', ' '))
            V.download_images(dest, urls=results.attrgot('contentUrl'))


def remove_failed_images(path: Path) -> L:
    image_paths = V.get_image_files(path)
    failed_images = V.verify_images(image_paths)
    return failed_images.map(Path.unlink)            

### Download data

In [ ]:
create_dataset(path, categories)
remove_failed_images(path)

### Create dataset

In [ ]:
data_block = V.DataBlock(
    blocks=(V.ImageBlock, V.CategoryBlock),
    get_items=V.get_image_files,
    splitter=V.RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=V.Resize(480),  # Presizing for single interpolation on GPU
    batch_tfms=V.aug_transforms(size=224, min_scale=0.75),
    get_y=V.parent_label,
)

### Create dataloaders

In [ ]:
dls = data_block.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

### Create model using transfer learning

In [ ]:
learn = V.cnn_learner(dls, V.resnet18, metrics=V.error_rate)

### Fine tune model

In [ ]:
learn.fine_tune(1)

### Confusion Matrix 

In [ ]:
interp = V.ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

### Top Losses

In [ ]:
interp.plot_top_losses(2, nrows=2)